In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from scipy import sparse
from scipy.interpolate import interp1d
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mticker
import matplotlib.cm as cm
from scipy.sparse import csr_matrix


cwd = os.getcwd()
model_path = os.path.abspath(os.path.join(cwd, os.pardir, 'models'))
sys.path.append(model_path)
from conference_model import *

In [ ]:
# Network
# Assuming 'SFHH_ConferenceNetwork.dat' is in the same directory and properly formatted
df = pd.read_csv('../data/SFHH_ConferenceNetwork.dat', delim_whitespace=True, header=None, names=['Timestamp', 'ID1', 'ID2'])

In [ ]:
# Parameters
s1 = 1/3.4
p_sev = 0.45
efficacy = 0.7
g_mild = 1/2.9
g_sev = 1/5.7
R0 = 2.8
t_interv = 0
t_end_conf = 5

prop_dict = {
    "Once daily": {"Mean": 0.78, "SD": 0.14},
    "Twice daily": {"Mean": 0.69, "SD": 0.15},
    "3 times a day": {"Mean": 0.64, "SD": 0.15},
    "4 times a day": {"Mean": 0.51, "SD": 0.20},
}

# Simulation setup
nr = 1000  # Number of realisations
Tmax = 50
dt = 0.01

In [ ]:
# Initial run for the untreated scenario (nr, dt, Tmax, A, s1, p_sev, g_mild, g_sev, efficacy, t_interv, R0, mean_prop, std_prop=None, t_end_conf=None, weighted='nonweighted', percentile=None):
_, _, IsimTot_untreated, cumul_mild_infections_untreated, cumul_sev_infections_untreated, cumul_infections_untreated, _, _, _, N = simulate_covid_spread(nr, dt, Tmax, df, s1, p_sev, g_mild, g_sev, 0, 0, R0, 0, t_end_conf=t_end_conf, weighted='weighted', percentile=90)



total_mild_infections_untreated = np.asarray([inner_array[-1] for inner_array in cumul_mild_infections_untreated])
total_sev_infections_untreated = np.asarray([inner_array[-1] for inner_array in cumul_sev_infections_untreated])

# Initialize a dictionary to store results
results = {}

# Loop through each scenario
for scenario, values in prop_dict.items():
    prop_mean = values['Mean']
    prop_std = values['SD']
    
    # Run simulation for the treated scenario
    _, _, IsimTot_treated, cumul_mild_infections_treated, cumul_sev_infections_treated, cumul_infections_treated, _, _, _, N = simulate_covid_spread(nr, dt, Tmax, df, s1, p_sev, g_mild, g_sev, efficacy, t_interv, R0, prop_mean, prop_std, t_end_conf=t_end_conf, weighted='weighted', percentile=90)


    # Calculate metrics for SEVERE infections and cumulative infections
    mean_infections = np.mean(IsimTot_treated, axis=0)
    lower_infections = np.percentile(IsimTot_treated, 2.5, axis=0)
    upper_infections = np.percentile(IsimTot_treated, 97.5, axis=0)
    mean_cumul_infections = np.mean(cumul_infections_treated, axis=0)
    lower_cumul_infections = np.percentile(cumul_infections_treated, 2.5, axis=0)
    upper_cumul_infections = np.percentile(cumul_infections_treated, 97.5, axis=0)
    
    # Calculate percentage of MILD infections avoided
    total_mild_infections_treated = np.asarray([inner_array[-1] for inner_array in cumul_mild_infections_treated])
    perc_mild_infections_avoided = 1 - total_mild_infections_treated / total_mild_infections_untreated
    mean_mild_averted = np.mean(perc_mild_infections_avoided)
    lower_mild_averted = np.percentile(perc_mild_infections_avoided, 2.5)
    upper_mild_averted = np.percentile(perc_mild_infections_avoided, 97.5)

    total_sev_infections_treated = np.asarray([inner_array[-1] for inner_array in cumul_sev_infections_treated])
    perc_sev_infections_avoided = 1 - total_sev_infections_treated / total_sev_infections_untreated
    mean_sev_averted = np.mean(perc_sev_infections_avoided)
    lower_sev_averted = np.percentile(perc_sev_infections_avoided, 2.5)
    upper_sev_averted = np.percentile(perc_sev_infections_avoided, 97.5)

    # Store results in the dictionary
    results[scenario] = {
        'Mean mild infections averted': mean_mild_averted,
        'Lower mild infections averted': lower_mild_averted,
        'Upper mild infections averted': upper_mild_averted,
        'Mean severe infections averted': mean_sev_averted,
        'Lower severe infections averted': lower_sev_averted,
        'Upper severe infections averted': upper_sev_averted,
        'Mean infections': mean_infections,
        'Lower infections': lower_infections,
        'Upper infections': upper_infections,
        'Mean cumul infections': mean_cumul_infections,
        'Lower cumul infections': lower_cumul_infections,
        'Upper cumul infections': upper_cumul_infections
    }


In [ ]:
plt.rcParams['font.family'] = 'Arial'
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))

# Data preparation for the first subplot
new_time_points = np.arange(0, Tmax, dt)
mean_untreated = np.mean(IsimTot_untreated, axis=0)
perc25_untreated = np.percentile(IsimTot_untreated, 2.5, axis=0)
perc975_untreated = np.percentile(IsimTot_untreated, 97.5, axis=0)

# Data preparation for the second subplot
mean_cumul_untreated = np.mean(cumul_infections_untreated, axis=0)
cumul_perc25_untreated = np.percentile(cumul_infections_untreated, 2.5, axis=0)
cumul_perc975_untreated = np.percentile(cumul_infections_untreated, 97.5, axis=0)

# Plot for untreated scenario - only once, in black
ax1.plot(new_time_points, mean_untreated, color='black', label='Mean untreated')
ax1.fill_between(new_time_points, perc25_untreated, perc975_untreated, color='black', alpha=0.2)
ax2.plot(new_time_points, mean_cumul_untreated, color='black', label='mean untreated')
ax2.fill_between(new_time_points, cumul_perc25_untreated, cumul_perc975_untreated, color='black', alpha=0.2)

# Define the color map and line styles
color_map = cm.get_cmap('viridis', 4)
line_styles = ['--', '-','-','-.']

# Loop through the first two treated scenarios only
for index, (scenario, values) in enumerate(results.items()):
    if index >= 1:  # Skip after the first two
        continue

    color = color_map(index+1)
    line_style = line_styles[index]

    # Data preparation from the stored results
    IsimTot_treated_mean = values['Mean infections']
    IsimTot_treated_perc25 = values['Lower infections']
    IsimTot_treated_perc975 = values['Upper infections']
    cumul_infections_treated_mean = values['Mean cumul infections']
    cumul_infections_treated_perc25 = values['Lower cumul infections']
    cumul_infections_treated_perc975 = values['Upper cumul infections']
    
    # Plot for the first subplot with different line styles
    ax1.plot(new_time_points, IsimTot_treated_mean, color=color, linestyle=line_style, label=f'Mean {scenario.lower()}')
    ax1.fill_between(new_time_points, IsimTot_treated_perc25, IsimTot_treated_perc975, color=color, alpha=0.2)
    
    # Plot for the second subplot with different line styles
    ax2.plot(new_time_points, cumul_infections_treated_mean, color=color, linestyle=line_style, label=f'Mean {scenario.lower()}')
    ax2.fill_between(new_time_points, cumul_infections_treated_perc25, cumul_infections_treated_perc975, color=color, alpha=0.2)

# Common plot adjustments
for ax in [ax1, ax2]:
    ax.axvline(x=t_end_conf, color='black', linestyle=':')
    ax.text(t_end_conf, 0.99, 'conference ends', ha='right', va='top', rotation=90, transform=ax.get_xaxis_transform())
    ax.set_xlabel('Day', weight='bold', fontsize=12)
    ax.xaxis.set_major_locator(mticker.MultipleLocator(5))

ax1.set_ylabel('No. infections', weight='bold', fontsize=12)
ax2.set_ylabel('Cumulative no. infections', weight='bold', fontsize=12)

ax1.fill_between([np.nan, np.nan], [np.nan, np.nan], color='black', alpha=0.2, label='95% confidence interval')
# Adding legends
ax1.legend()

plt.tight_layout()

# Define the directory path and the filename
directory = '../figures'
filename = 'figure4e.svg'

# Check if the directory exists
if not os.path.exists(directory):
    # If the directory does not exist, create it
    os.makedirs(directory)

# Save the figure
plt.savefig(os.path.join(directory, filename), bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['font.family'] = 'Arial'
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 3.5))  # Create three subplots without sharing y-axis

labels = []
x_index = 1

# Collecting all y-values to determine common y-axis limits
all_y_values = []

# First subplot for treatment adherence (ax1)
for scenario, values in prop_dict.items():
    mean_adherence = values['Mean']  # Assuming 'Mean' represents mean adherence
    sd_adherence = values['SD']  # Assuming 'SD' represents standard deviation
    all_y_values.extend([mean_adherence - sd_adherence, mean_adherence + sd_adherence])

    ax1.plot([x_index, x_index], [mean_adherence - sd_adherence, mean_adherence + sd_adherence], '-k')
    ax1.plot(x_index, mean_adherence, 'ko')
    labels.append(scenario)
    x_index += 1

ax1.set_xticks(range(1, len(labels) + 1))
ax1.set_xticklabels(labels)
ax1.set_ylabel('Treatment adherence (%)')
ax1.yaxis.set_major_formatter(PercentFormatter(1, decimals=0))

# Second subplot for mild infections (ax2)
x_index = 1  # Reset x_index for the next subplot
for scenario, values in results.items():
    mean_averted_mild = max(0, values['Mean mild infections averted'])
    lower_averted_mild = max(0, values['Lower mild infections averted'])
    upper_averted_mild = max(0, values['Upper mild infections averted'])
    all_y_values.extend([lower_averted_mild, upper_averted_mild])

    ax2.plot([x_index, x_index], [lower_averted_mild, upper_averted_mild], '-k')
    ax2.plot(x_index, mean_averted_mild, 'ko')
    x_index += 1

ax2.set_xticks(range(1, len(labels) + 1))
ax2.set_xticklabels(labels)
ax2.set_ylabel('Mild infections averted (%)')
ax2.yaxis.set_major_formatter(PercentFormatter(1, decimals=0))

# Third subplot for severe infections (ax3)
x_index = 1  # Reset x_index for the next subplot
for scenario, values in results.items():
    mean_averted_severe = max(0, values['Mean severe infections averted'])
    lower_averted_severe = max(0, values['Lower severe infections averted'])
    upper_averted_severe = max(0, values['Upper severe infections averted'])
    all_y_values.extend([lower_averted_severe, upper_averted_severe])

    ax3.plot([x_index, x_index], [lower_averted_severe, upper_averted_severe], '-k')
    ax3.plot(x_index, mean_averted_severe, 'ko')
    x_index += 1

ax3.set_xticks(range(1, len(labels) + 1))
ax3.set_xticklabels(labels)
ax3.set_ylabel('Severe infections averted (%)')
ax3.yaxis.set_major_formatter(PercentFormatter(1, decimals=0))

# Synchronize y-axis limits
min_y, max_y = min(all_y_values), max(all_y_values)
ax1.set_ylim(-0.05, 1.05)
ax2.set_ylim(-0.05, 1.05)
ax3.set_ylim(-0.05, 1.05)

plt.tight_layout()

# Define the directory path and the filename
directory = '../figures'
filename = 'figure4f-h.svg'

# Check if the directory exists
if not os.path.exists(directory):
    # If the directory does not exist, create it
    os.makedirs(directory)

# Save the figure
plt.savefig(os.path.join(directory, filename), bbox_inches='tight')

plt.show()